In [1]:
%run mainTNR.py

RGstep:  0  , Truncation Errors: [2.008217989894428e-06, 4.710099382075902e-05, 4.3385250754765434e-05, 0.00010246157077446297]
time spent for this RGstep:  0.8481032848358154
shape of A: (6, 6, 6, 6)
shape of q: (4, 4, 6)
shape of s: (6, 4)
shape of u: (4, 4, 4, 4)
shape of y: (4, 4, 4)
shape of v: (4, 4, 6)
shape of w: (4, 4, 6)
RGstep:  1  , Truncation Errors: [1.2052261389767233e-06, 5.5314341863221894e-05, 8.854347959034827e-05, 0.00011995394719820866]
time spent for this RGstep:  0.8429150581359863
shape of A: (6, 6, 6, 6)
shape of q: (6, 6, 6)
shape of s: (6, 4)
shape of u: (6, 6, 4, 4)
shape of y: (6, 4, 4)
shape of v: (6, 6, 6)
shape of w: (4, 4, 6)
RGstep:  2  , Truncation Errors: [1.3926494470403928e-06, 6.887552980815456e-05, 8.107328747758827e-05, 0.00010534040724117942]
time spent for this RGstep:  0.7530243396759033
shape of A: (6, 6, 6, 6)
shape of q: (6, 6, 6)
shape of s: (6, 4)
shape of u: (6, 6, 4, 4)
shape of y: (6, 4, 4)
shape of v: (6, 6, 6)
shape of w: (4, 4, 6)


In [1]:
%run mainScInvTNR.py

Iteration:  0  of  2000 , Trunc. Error: 6.08743e-10,4.06838e-06
Iteration:  100  of  2000 , Trunc. Error: 6.08743e-10,2.94729e-06
Iteration:  200  of  2000 , Trunc. Error: 6.08743e-10,2.48122e-06
Iteration:  300  of  2000 , Trunc. Error: 6.08743e-10,2.1954e-06
Iteration:  400  of  2000 , Trunc. Error: 6.08743e-10,1.98306e-06
Iteration:  500  of  2000 , Trunc. Error: 6.08743e-10,1.81134e-06
Iteration:  600  of  2000 , Trunc. Error: 6.08743e-10,1.67127e-06
Iteration:  700  of  2000 , Trunc. Error: 6.08743e-10,1.55939e-06
Iteration:  800  of  2000 , Trunc. Error: 6.08743e-10,1.4727e-06
Iteration:  900  of  2000 , Trunc. Error: 6.08743e-10,1.40773e-06
Iteration:  1000  of  2000 , Trunc. Error: 6.08743e-10,1.36077e-06
Iteration:  1100  of  2000 , Trunc. Error: 6.08743e-10,1.32807e-06
Iteration:  1200  of  2000 , Trunc. Error: 6.08743e-10,1.30612e-06
Iteration:  1300  of  2000 , Trunc. Error: 6.08743e-10,1.29188e-06
Iteration:  1400  of  2000 , Trunc. Error: 6.08743e-10,1.2829e-06
RGstep: 0 